<a href="https://colab.research.google.com/github/UICHEOL-HWANG/colab_upload/blob/master/dataq_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
import os
import sys
import urllib.request
import datetime
import time
import json
import math
import pandas as pd
from urllib.parse import urlencode, unquote
from xml_to_dict import XMLtoDict
import matplotlib.pyplot as plt
import koreanize_matplotlib

import getpass

servicekey = getpass.getpass("비밀번호 입력>>")
xd = XMLtoDict()

비밀번호 입력>>··········


In [94]:
pip install koreanize_matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 65.9 MB/s eta 0:00:00


In [6]:
pip install xml_to_dict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [185]:
def get_reQuest_url(url):
  req = urllib.request.Request(url)
  try:
    response = urllib.request.urlopen(req)
    if response.getcode() == 200:
      print ("[%s] Url Request Success" % datetime.datetime.now())
      return response.read().decode('utf-8')
  except Exception as e:
    print(e)
    print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
  return None

def getTouristWrite(yyyymm, national_code, ed_cd):

  service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/"
  parameters = "getEdrcntTourismStatsList?serviceKey=" + servicekey   # 인증키
  parameters += "&YM=" + yyyymm
  parameters += "&NAT_CD=" + national_code
  parameters += "&ED_CD=" + ed_cd

  url = service_url + parameters
  print(url)   # 액세스 거부 여부 확인용 출력
  responseDecode = get_reQuest_url(url)   # [CODE 1]

  if(responseDecode == None):
    return None
  else :
    return xd.parse(responseDecode)

def getTouristInfo(nat_cd,ed_cd,nStartYear,nEndYear):
  xd_result = []
  result = []
  natName = ''
  dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12))
  isDataEnd = 0

  for year in range(nStartYear,nEndYear+1):
    for month in range(1,13):
      if(isDataEnd ==1 ): break
      yyyymm = "{0}{1:0>2}".format(str(year), str(month))
      xmlData = getTouristWrite(yyyymm, nat_cd, ed_cd)
      if(xmlData['response']['header']['resultMsg']== 'OK'):
        if xmlData['response']['body']['items'] == '':
          isDataEnd = 1
          dataEND = "{0}{1:0>2}".format(str(year),str(month-1))
          print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))
          break
        json.dumps(xmlData,indent=4,sort_keys=True, ensure_ascii=False)
        natName = xmlData['response']['body']['items']['item']['natKorNm']
        natName = natName.replace(' ','')
        num = xmlData['response']['body']['items']['item']['num']
        ed = xmlData['response']['body']['items']['item']['ed']

        xd_result.append({'natName':natName,'nat_cd':nat_cd,'yyyymm':yyyymm,'visit_cnt':num})
        result.append([natName, nat_cd, yyyymm, num])
  return (xd_result,result, natName, ed, dataEND)

def main():
  xd_result = []
  result = []
  print('국내 입국한 외국인들의 통계 데이터 입력중')
  nat_cd = input("국가 코드 입력 >>")
  nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
  nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
  ed_cd = "E"
  xd_result,result,natName,ed,dataEND = getTouristInfo(nat_cd,ed_cd,nStartYear,nEndYear)

  columns = ['입국자국가','국가코드','입국연월','입국자 수']
  df = pd.DataFrame(result,columns=columns)
  df['입국연월'] = df['입국연월'].apply(lambda x : x[0:4] + '-' +x[4:6])
  df['입국자 수'] = df['입국자 수'].astype(int)

  #그래프 그리기
  plt.rcParams["font.size"] = 12
  plt.figure(figsize =(20,10))
  df.plot(kind='line',x = '입국연월',y='입국자 수',grid=True)
  plt.show()


In [ ]:
main() # API KEY 파싱이 개인정보 문제가 있어 출력하지 못했음